In [1]:
import sys
sys.path.insert(0, '..')

In [3]:
from pymr import heart as pyheart
import numpy as np
import nibabel

heart = nibabel.load(r"test_gt.nii.gz").get_fdata()


In [ ]:
def get_frame(heart_mask):
    if heart_mask.shape.size == 3: #xyt
        curve = np.sum(heart_mask==1, axis=(0, 1))
    elif heart_mask.shape.size == 4: #xyzt
        curve = np.sum(heart_mask==1, axis=(0, 1, 2))
    else:
        sys_frame = -1
        dia_frame = -1
        return sys_frame, dia_frame

    curve = curve.astype(np.float)
    dia_frame = np.argmax(curve)
    curve[curve==0] = 1e20
    sys_frame = np.argmin(curve)

    return sys_frame, dia_frame

In [ ]:
%matplotlib inline
from scipy.io import loadmat
from os.path import basename
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from pymr.heart import ahaseg

In [ ]:
from .heart.cine import convert

In [ ]:
curve = np.sum(heart==1, axis=(0, 1, 2))
dia_frame = np.argmax(curve)
curve[curve==0] = 1e8
sys_frame = np.argmin(curve)

In [ ]:
#step 1: collect dia_frame and sys_frame
curve = np.sum(heart, axis=(0, 1, 2))
dia_frame = np.argmax(curve)
curve[curve==0] = 1e8
sys_frame = np.argmin(curve)

#step 2: 


In [ ]:
def get_slice_label(heart4d):
    heart = heart4d.copy()
    curve3 = np.sum(heart==3, axis=(0, 1, 3))
    curve2 = np.sum(heart==2, axis=(0, 1, 3))
    curve1 = np.sum(heart==1, axis=(0, 1, 3))

    curve_and = (curve3 > 0) & (curve2 > 0) & (curve1 > 0)
    curve_or = (curve3 > 0) | (curve2 > 0) | (curve1 > 0)
    curve_and = curve_and.astype(np.int)
    curve_or = curve_or.astype(np.int)
    curve_diff = curve_or-curve_and
    slice_label = curve_and * 0
    curve_apex = curve_diff.copy()
    curve_apex[:curve_apex.size//2] = 0
    curve_basal = curve_diff.copy()
    curve_basal[curve_apex.size//2:] = 0
    #curve_apex = curve_apex * 4
    
    loc = dict()
    if np.sum(curve_apex) > 0:
        #got_apex = True    
        loc[3] = [1]*1 + [2]*1 + [3]*1
        loc[4] = [1]*1 + [2]*2 + [3]*1
        loc[5] = [1]*2 + [2]*2 + [3]*1
        loc[6] = [1]*2 + [2]*2 + [3]*2
        loc[7] = [1]*2 + [2]*3 + [3]*2
        loc[8] = [1]*3 + [2]*3 + [3]*2
        loc[9] = [1]*3 + [2]*3 + [3]*3
        loc[10] = [1]*3 + [2]*4 + [3]*3
        loc[11] = [1]*4 + [2]*4 + [3]*3
        loc[12] = [1]*4 + [2]*4 + [3]*4
        loc[13] = [1]*4 + [2]*5 + [3]*4
        loc[14] = [1]*5 + [2]*5 + [3]*4
        loc[15] = [1]*5 + [2]*5 + [3]*5
    else:
        loc[3] = [1]*1 + [2]*1 + [3]*1
        loc[4] = [1]*1 + [2]*1 + [3]*1 + [4]*1
        loc[5] = [1]*1 + [2]*2 + [3]*1 + [4]*1
        loc[6] = [1]*2 + [2]*2 + [3]*1 + [4]*1
        loc[7] = [1]*2 + [2]*2 + [3]*2 + [4]*1
        loc[8] = [1]*2 + [2]*3 + [3]*2 + [4]*1
        loc[9] = [1]*3 + [2]*3 + [3]*2 + [4]*1
        loc[10] = [1]*3 + [2]*3 + [3]*3 + [4]*1
        loc[11] = [1]*3 + [2]*3 + [3]*3 + [4]*2
        loc[12] = [1]*3 + [2]*4 + [3]*3 + [4]*2
        loc[13] = [1]*4 + [2]*4 + [3]*3 + [4]*2
        loc[14] = [1]*4 + [2]*4 + [3]*4 + [4]*2
        loc[15] = [1]*4 + [2]*4 + [3]*4 + [4]*3
        
    slice_label[curve_and > 0] = loc[np.sum(curve_and)]
    slice_label[curve_apex > 0] = 4
    slice_label[curve_basal > 0] = 5
    return slice_label


In [ ]:
slice_label = get_slice_label(heart)
print(slice_label)
offset = dict()
offset[1] = 0
offset[2] = 6
offset[3] = 12

count = -1
heart_aha17_4d = heart  * 0
for ii in range(slice_label.size):
    #print(slice_label[ii])
    count = count + 1
    if (slice_label[ii] == 0) or (slice_label[ii] == 5):
        continue
        
    if slice_label[ii] == 4:
        temp = heart[:, :, ii, :].copy()
        temp[temp==2] = 17
        temp[temp==1] = 0
        temp[temp==3] = 0
        heart_aha17_4d[:, :, ii, :] = temp
        

    heart_xyt = heart[:, :, ii, :].copy()
    
    curve = np.sum(heart_xyt, axis=(0, 1))
    dia_frame = np.argmax(curve)
    curve[curve==0] = 1e20
    sys_frame = np.argmin(curve)
    #print(dia_frame, sys_frame)
    heart_xy_dia = heart_xyt[..., dia_frame]
    heart_xy_sys = heart_xyt[..., sys_frame]
    if slice_label[ii] == 3:
        nseg = 4
    else:
        nseg = 6
    
    if (np.sum(heart_xy_dia==3)) == 0 or (np.sum(heart_xy_sys==3) == 0):
        slice_label[ii] = 5
        continue
        
        
    dia_seg = ahaseg.get_seg((heart_xy_dia==1, heart_xy_dia==2, heart_xy_dia==3), nseg)
    sys_seg = ahaseg.get_seg((heart_xy_sys==1, heart_xy_sys==2, heart_xy_sys==3), nseg)
    
    dia_seg[dia_seg > 0] = dia_seg[dia_seg > 0] + offset[slice_label[ii]]
    sys_seg[sys_seg > 0] = sys_seg[sys_seg > 0] + offset[slice_label[ii]]
    
    heart_aha17_4d[:, :, ii, dia_frame] = dia_seg
    heart_aha17_4d[:, :, ii, sys_frame] = sys_seg


In [ ]:
plt.figure()
plt.imshow(heart_aha17_4d[:, :, 5, dia_frame], vmin=0, vmax=17)
plt.colorbar()
plt.figure()
plt.imshow(heart_aha17_4d[:, :, 8, dia_frame], vmin=0, vmax=17)
plt.colorbar()
plt.figure()
plt.imshow(heart_aha17_4d[:, :, 10, dia_frame], vmin=0, vmax=17)
plt.colorbar()

plt.figure()
plt.imshow(heart_aha17_4d[:, :, 11, dia_frame], vmin=0, vmax=17)
plt.colorbar()

In [ ]:
slice_label[curve_and > 0] = loc[np.sum(curve_and)]
slice_label[curve_apex > 0] = 4
slice_label[curve_basal > 0] = 5

In [ ]:
heart_mask = heart[..., 7, sys_frame]
LVbmask = (heart_mask==1)
LVwmask = (heart_mask==2)
RVbmask = (heart_mask==3)
heart_mask = (LVbmask, LVwmask, RVbmask)
heart_seg_sys = ahaseg.get_seg(heart_mask, nseg=6)
plt.imshow(heart_seg_sys)

In [ ]:
heart_mask = heart[..., 7, dia_frame]
LVbmask = (heart_mask==1)
LVwmask = (heart_mask==2)
RVbmask = (heart_mask==3)
heart_mask = (LVbmask, LVwmask, RVbmask)
heart_seg_dia = ahaseg.get_seg(heart_mask, nseg=6)
plt.imshow(heart_seg_dia)

In [ ]:
def get_center(label_mask):
    center_list = []
    for ii in range(np.max(label_mask)):
        center = ndimage.center_of_mass(label_mask==(ii+1))
        center_list.append(center)
        
    return np.array(center_list)

In [ ]:
sys6 = get_center(heart_seg_sys)
dia6 = get_center(heart_seg_dia)

In [ ]:
plt.scatter(dia6[:, 1], dia6[:, 0])
plt.scatter(sys6[:, 1], sys6[:, 0])

In [ ]:
vector = (sys6 - dia6)

In [ ]:
vector

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(heart_seg_dia, cmap='gray')
index = np.arange(6)
plt.quiver(dia6[index, 1], dia6[index, 0] , 
           vector[index, 1], vector[index, 0]*-1,
           color='r', headlength=6)
plt.axis
#plt.scatter(dia6[index, 1], dia6[index, 0])
#plt.scatter(sys6[index, 1], sys6[index, 0])
#plt.scatter(dia6[:, 1], dia6[:, 0])
#plt.ylim(0, heart_seg_sys.shape[1])
#plt.xlim(0, heart_seg_sys.shape[0])

In [ ]:
(dia6[index, 1], dia6[index, 0])

In [ ]:
(sys6[index, 1], sys6[index, 0])

In [ ]:
(vector[index, 1], vector[index, 0])

In [ ]:
!pip install dipy

In [ ]:
import numpy as np
from dipy.align.imwarp import SymmetricDiffeomorphicRegistration
from dipy.align.metrics import SSDMetric, CCMetric
import dipy.align.imwarp as imwarp
#from dipy.data import get_fnames
#from dipy.io.image import load_nifti_data
#from dipy.segment.mask import median_otsu
from dipy.viz import regtools

moving = heart_seg_dia
static = heart_seg_sys

sigma_diff = 3.0
radius = 4
metric = CCMetric(2, sigma_diff, radius)
dim = static.ndim
metric = SSDMetric(dim)
"""
Let's use a scale space of 3 levels
"""

level_iters = [100, 50, 25]
sdr = SymmetricDiffeomorphicRegistration(metric, level_iters)
#sdr.callback = callback_CC

"""
And execute the optimization
"""

mapping = sdr.optimize(static, moving)

warped = mapping.transform(moving)

"""
We can see the effect of the warping by switching between the images before and
after registration
"""

regtools.overlay_images(static, moving, 'Static', 'Overlay', 'Moving',
                        't1_slices_input.png')

regtools.overlay_images(static, warped, 'Static', 'Overlay', 'Warped moving',
                        't1_slices_res.png')


In [ ]:
regtools.plot_2d_diffeomorphic_map(mapping, 5, 'diffeomorphic_map_b0s.png')

In [ ]:
backward_field = mapping.get_backward_field()

In [ ]:
plt.quiver(backward_field[..., 0], backward_field[..., 1])

In [ ]:
np.max(backward_field)

In [ ]:
plt.imshow(backward_field[..., 0]**2 + backward_field[..., 1]**2)
plt.quiver(backward_field[..., 0], backward_field[..., 1])

In [ ]:
v = np.sqrt(backward_field[..., 0]**2 + backward_field[..., 1]**2)

In [ ]:
plt.imshow(v)

In [ ]:
plt.imshow(heart_seg_sys)

In [ ]:
dynamic_png(np.arange(1, 18))

In [ ]:

def plot_aha17(aha17, vmin=None, vmax=None, fname=None):
    import numpy as np
    import matplotlib as mpl
    import matplotlib.pyplot as plt

    if vmin is None:
        vmin = 0
        vmax = np.max(aha17)
    
        #Create the fake data
    #data = np.array(range(17))*10 + 1


    # Make a figure and axes with dimensions as desired.
    fig=plt.figure(figsize=(9,8), dpi=200)

    #fig, ax = plt.subplots(figsize=(8,12),nrows=1, ncols=1,
    #                       subplot_kw=dict(projection='polar'))
    #fig.canvas.set_window_title('Left Ventricle Bulls Eyes (AHA)')

    # Create the axis for the colorbars
    ax = fig.add_axes([0.1, 1-8/9.+0.1, 8./9.*0.7, 1*0.7],projection='polar')
    axl = fig.add_axes([0.76, 0.3, 0.03, 0.5])
    #axl = fig.add_axes([0.76, 0.3, 0.01, 0.5])
    #axl.set_aspect(1)
    #axl2 = fig.add_axes([0.41, 0.15, 0.2, 0.05])
    #axl3 = fig.add_axes([0.69, 0.15, 0.2, 0.05])


    # Set the colormap and norm to correspond to the data for which
    # the colorbar will be used.

    cmap = mpl.cm.jet
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    #norm = mpl.colors.Normalize(vmin=0, vmax=256)
    # ColorbarBase derives from ScalarMappable and puts a colorbar
    # in a specified axes, so it has everything needed for a
    # standalone colorbar.  There are many more kwargs, but the
    # following gives a basic continuous colorbar with ticks
    # and labels.
    cb1 = mpl.colorbar.ColorbarBase(axl, cmap=cmap, norm=norm,
                                    orientation='vertical')
    #cb1.set_label('A. U.')


    # Create the 17 segment model
    bullseye_plot(ax, aha17, cmap=cmap, norm=norm)

    #ax.set_title('Bulls Eye (AHA)')

    #plt.clf()

    plt.show()
    if fname is not None:
        plt.savefig(fname)


def bullseye_plot(ax, data, segBold=[], cmap=None, norm=None):
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    """
    Bullseye representation for the left ventricle.

    Parameters
    ------------
    ax : axes
    data : list of int and float
        The intensity values for each of the 17 segments
    cmap : ColorMap or None
        Optional argument to set the disaried colormap
    norm : Normalize or None
        Optional argument to normalize data into the [0.0, 1.0] range
    segBold: list of int
        A list with the segments to highlight


    Notes
    -----
    This function create the 17 segment model for the left ventricle according
    to the American Heart Association (AHA) [1]_

    References
    ----------
    .. [1] M. D. Cerqueira, N. J. Weissman, V. Dilsizian, A. K. Jacobs,
        S. Kaul, W. K. Laskey, D. J. Pennell, J. A. Rumberger, T. Ryan,
        and M. S. Verani, "Standardized myocardial segmentation and nomenclature
        for tomographic imaging of the heart", Circulation, vol. 105, no. 4,
        pp. 539-542, 2002.
    """

    linewidth = 2
    data = np.array(data).ravel()

    if cmap is None:
        cmap = plt.cm.jet

    if norm is None:
        norm = mpl.colors.Normalize(vmin=data.min(), vmax=data.max())

    theta = np.linspace(0, 2*np.pi, 768)
    r = np.linspace(0.2, 1, 4)

    # Create the bound for the segment 17
    for i in range(r.shape[0]):
        ax.plot(theta, np.repeat(r[i], theta.shape), '-k', lw=linewidth)

    # Create the bounds for the segments  1-12
    for i in range(6):
        theta_i = i*60*np.pi/180
        ax.plot([theta_i, theta_i], [r[1], 1], '-k', lw=linewidth)

    # Create the bounds for the segmentss 13-16
    for i in range(4):
        theta_i = i*90*np.pi/180 - 45*np.pi/180
        ax.plot([theta_i, theta_i], [r[0], r[1]], '-k', lw=linewidth)

    # Fill the segments 1-6
    r0 = r[2:4]
    r0 = np.repeat(r0[:, np.newaxis], 128, axis=1).T
    for i in range(6):
        # First segment start at 60 degrees
        theta0 = theta[i*128:i*128+128] + 60*np.pi/180
        theta0 = np.repeat(theta0[:, np.newaxis], 2, axis=1)
        z = np.ones((128, 2))*data[i]
        ax.pcolormesh(theta0, r0, z, cmap=cmap, norm=norm)
        if i+1 in segBold:
            ax.plot(theta0, r0, '-k', lw=linewidth+2)
            ax.plot(theta0[0], [r[2], r[3]], '-k', lw=linewidth+1)
            ax.plot(theta0[-1], [r[2], r[3]], '-k', lw=linewidth+1)

    # Fill the segments 7-12
    r0 = r[1:3]
    r0 = np.repeat(r0[:, np.newaxis], 128, axis=1).T
    for i in range(6):
        # First segment start at 60 degrees
        theta0 = theta[i*128:i*128+128] + 60*np.pi/180
        theta0 = np.repeat(theta0[:, np.newaxis], 2, axis=1)
        z = np.ones((128, 2))*data[i+6]
        ax.pcolormesh(theta0, r0, z, cmap=cmap, norm=norm)
        if i+7 in segBold:
            ax.plot(theta0, r0, '-k', lw=linewidth+2)
            ax.plot(theta0[0], [r[1], r[2]], '-k', lw=linewidth+1)
            ax.plot(theta0[-1], [r[1], r[2]], '-k', lw=linewidth+1)

    # Fill the segments 13-16
    r0 = r[0:2]
    r0 = np.repeat(r0[:, np.newaxis], 192, axis=1).T
    for i in range(4):
        # First segment start at 45 degrees
        theta0 = theta[i*192:i*192+192] + 45*np.pi/180
        theta0 = np.repeat(theta0[:, np.newaxis], 2, axis=1)
        z = np.ones((192, 2))*data[i+12]
        ax.pcolormesh(theta0, r0, z, cmap=cmap, norm=norm)
        if i+13 in segBold:
            ax.plot(theta0, r0, '-k', lw=linewidth+2)
            ax.plot(theta0[0], [r[0], r[1]], '-k', lw=linewidth+1)
            ax.plot(theta0[-1], [r[0], r[1]], '-k', lw=linewidth+1)

    #Fill the segments 17
    if data.size == 17:
        r0 = np.array([0, r[0]])
        r0 = np.repeat(r0[:, np.newaxis], theta.size, axis=1).T
        theta0 = np.repeat(theta[:, np.newaxis], 2, axis=1)
        z = np.ones((theta.size, 2))*data[16]
        ax.pcolormesh(theta0, r0, z, cmap=cmap, norm=norm)
        if 17 in segBold:
            ax.plot(theta0, r0, '-k', lw=linewidth+2)

    ax.set_ylim([0, 1])
    ax.set_yticklabels([])
    ax.set_xticklabels([])


In [ ]:
%matplotlib inline
plot_aha17(np.arange(1, 18))

In [ ]:
plt.show